# 原图json：

# anchor-detr & dn-detr

In [22]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# ==================================================
# 配置
# ==================================================
VERSION = "ori"
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco/annotations/instances_val2017.json"

# 模型结果路径列表
MODEL_RESULTS = [
    # f"{BASE_DIR}/save/attack/detection/detr/ori/res.json",
    # f"{BASE_DIR}/save/attack/detection/deformable-detr/ori/res.json",
    # f"{BASE_DIR}/save/attack/detection/sparse-detr/ori/res.json",
    f"{BASE_DIR}/save/attack/detection/anchor-detr/ori/res.json",
    f"{BASE_DIR}/save/attack/detection/dn-detr/ori/res.json",
]

# ==================================================
# 辅助函数
# ==================================================
def load_image_mapping(annotations_path):
    """加载 image_id → file_name 映射"""
    with open(annotations_path, "r") as f:
        data = json.load(f)
    return {img["id"]: img["file_name"] for img in data["images"]}

def convert_to_standard_format(input_path, image_mapping, version):
    """将原始 JSON 转换为标准格式"""
    if not os.path.exists(input_path):
        print(f"⚠️ 文件不存在: {input_path}")
        return
    model_name = input_path.split("/")[-3]
    with open(input_path, "r") as f:
        data = json.load(f)

    # 如果是 dict（比如包含 "annotations"）
    if isinstance(data, dict):
        detections = data.get("annotations", [])
    else:
        detections = data

    results = []
    for det in detections:
        if det.get("category_id") != 1:
            continue

        image_id = int(det.get("image_id", -1))
        file_name = image_mapping.get(image_id, f"unknown_{image_id}.png")
        bbox = det.get("bbox", [])
        if len(bbox) == 4:
            # 自动判断是否 xywh
            if bbox[2] < bbox[0] or bbox[3] < bbox[1]:
                x, y, w, h = bbox
                bbox = [x, y, x + w, y + h]
        score = float(det.get("score", 0.0))
        area = abs((bbox[2]-bbox[0]) * (bbox[3]-bbox[1]))
        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": 1,
            "category_name": "person",
            "bbox": bbox,
            "score": score,
            "area": area,
            "model": model_name,
            "version": version
        })
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    output_path = os.path.join(os.path.dirname(input_path), "res-std.json")
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    # --- 新增功能：打印前 30 行内容 ---
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 30 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            # 打印最多 30 行
            for i, line in enumerate(lines[:30]):
                # 移除行末的换行符
                print(line.strip('\n'))
            if len(lines) > 30:
                print("... (已省略更多行)")
                
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")

# ==================================================
# 主程序
# ==================================================
def main():
    image_mapping = load_image_mapping(ANNOT_PATH)
    for path in tqdm(MODEL_RESULTS, desc="标准化转换中"):
        convert_to_standard_format(path, image_mapping, VERSION)
    print("\n✅ 全部模型 JSON 已转换为标准格式。")

if __name__ == "__main__":
    main()


标准化转换中: 100%|██████████| 2/2 [00:00<00:00,  4.87it/s]

✅ anchor-detr: 4843 条记录 → /opt/data/private/BlackBox/save/attack/detection/anchor-detr/ori/res-std.json
--------------------------------------------------
📄 anchor-detr - res-std.json 前 30 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      397.2433166503906,
      215.4185333251953,
      567.6422119140625,
      711.1751861572266
    ],
    "score": 0.8803859353065491,
    "area": 84476.38596219383,
    "model": "anchor-detr",
    "version": "ori"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      187.16326904296875,
      459.8171691894531,
      235.64108276367188,
      501.47100830078125
    ],
    "score": 0.47865647077560425,
    "area": 2019.287053191103,
... (已省略更多行)
--------------------------------------------------
✅ dn-detr: 1439 条记录 → /opt/data/private/BlackBox/save/attack/detection/dn-detr/ori/res-std.j

# sparse-detr

In [23]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# =========================
# 配置
# =========================
VERSION = "ori"  # 原图/无补丁；若是补丁版可写成 "patch-<ver>"
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco/annotations/instances_val2017.json"

# Sparse-DETR 原始结果（未标准化）路径
INPUT_PATH = f"{BASE_DIR}/save/attack/detection/sparse-detr/ori/res.json"

# =========================
# 辅助函数
# =========================
def load_image_mapping(annotations_path):
    """加载 image_id → file_name 映射"""
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return {img["id"]: img["file_name"] for img in data["images"]}

def coco_category_name(cat_id: int) -> str:
    """常见 COCO 类别名映射（不足部分用 cls_<id> 回退）"""
    m = {
        1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane",
        6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light",
        11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench",
        16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow",
        22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
        27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase",
        # …可按需补充
    }
    return m.get(cat_id, f"cls_{cat_id}")

def xywh_to_xyxy(b):
    """将 [x, y, w, h] → [x1, y1, x2, y2]（Sparse-DETR 结果为 xywh）"""
    if not isinstance(b, (list, tuple)) or len(b) != 4:
        return b
    x, y, w, h = b
    return [float(x), float(y), float(x + w), float(y + h)]

def save_preview(model_name, output_path, max_lines=50):
    """打印前 max_lines 行，便于快速校验"""
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 {max_lines} 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for i, line in enumerate(lines[:max_lines]):
                print(line.rstrip("\n"))
            if len(lines) > max_lines:
                print("... (已省略更多行)")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")

# =========================
# 主转换
# =========================
def main():
    print("=== Sparse-DETR → 标准化转换开始 ===")
    if not os.path.exists(INPUT_PATH):
        print(f"⚠️ 未找到输入文件: {INPUT_PATH}")
        return

    image_mapping = load_image_mapping(ANNOT_PATH)

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Sparse-DETR 的 res.json 是 list[det]，每个 det:
    # {image_id, category_id, bbox=[x,y,w,h], score}
    detections = data if isinstance(data, list) else data.get("annotations", [])
    model_name = "sparse-detr"

    results = []
    for det in tqdm(detections, desc="标准化中"):
        try:
            image_id = int(det.get("image_id"))
        except Exception:
            continue

        file_name = image_mapping.get(image_id, f"unknown_{image_id}.png")
        cat_id = int(det.get("category_id", -1))
        cat_name = coco_category_name(cat_id)
        bbox_xyxy = xywh_to_xyxy(det.get("bbox", []))
        score = float(det.get("score", 0.0))

        if not isinstance(bbox_xyxy, list) or len(bbox_xyxy) != 4:
            continue

        x1, y1, x2, y2 = bbox_xyxy
        area = abs((x2 - x1) * (y2 - y1))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": cat_id,
            "category_name": cat_name,
            "bbox": [x1, y1, x2, y2],
            "score": score,
            "area": area,
            "model": model_name,
            "version": VERSION
        })
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    output_path = os.path.join(os.path.dirname(INPUT_PATH), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    save_preview(model_name, output_path)
    print("=== 完成 ===")

if __name__ == "__main__":
    main()


=== Sparse-DETR → 标准化转换开始 ===


标准化中: 100%|██████████| 28800/28800 [00:00<00:00, 211392.39it/s]


✅ sparse-detr: 28800 条记录 → /opt/data/private/BlackBox/save/attack/detection/sparse-detr/ori/res-std.json
--------------------------------------------------
📄 sparse-detr - res-std.json 前 50 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      404.661,
      219.672,
      563.142,
      703.506
    ],
    "score": 0.92587,
    "area": 76678.49615400002,
    "model": "sparse-detr",
    "version": "ori"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      195.753,
      460.113,
      250.43599999999998,
      507.335
    ],
    "score": 0.48012,
    "area": 2582.2406259999984,
    "model": "sparse-detr",
    "version": "ori"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      87.793,
      455.684,
      156.376,
      514.0160000000001
  

# deformable-detr

In [24]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# ==================================================
# 配置
# ==================================================
VERSION = "ori"  # 原图版本（clean）
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco/annotations/instances_val2017.json"

# Deformable-DETR 原始检测结果路径
INPUT_PATH = f"{BASE_DIR}/save/attack/detection/deformable-detr/ori/res.json"

# ==================================================
# 辅助函数
# ==================================================
def load_image_mapping(annotations_path):
    """加载 image_id → file_name 映射"""
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return {img["id"]: img["file_name"] for img in data["images"]}

def coco_category_name(cat_id: int) -> str:
    """COCO 类别名映射（可按需扩展）"""
    names = {
        1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane",
        6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light",
        11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench",
        16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow",
        22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
        27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase",
    }
    return names.get(cat_id, f"cls_{cat_id}")

def save_preview(model_name, output_path, max_lines=30):
    """打印前 max_lines 行 JSON 内容，供人工快速检查"""
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 {max_lines} 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for i, line in enumerate(lines[:max_lines]):
                print(line.rstrip("\n"))
            if len(lines) > max_lines:
                print("... (已省略更多行)")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")

# ==================================================
# 主转换
# ==================================================
def main():
    print("=== Deformable-DETR → 标准化转换开始 ===")

    if not os.path.exists(INPUT_PATH):
        print(f"⚠️ 未找到输入文件: {INPUT_PATH}")
        return

    image_mapping = load_image_mapping(ANNOT_PATH)

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 原始数据通常是 list[dict]
    detections = data if isinstance(data, list) else data.get("annotations", [])
    model_name = "deformable-detr"

    results = []
    for det in tqdm(detections, desc="标准化中"):
        try:
            image_id = int(det.get("image_id"))
        except Exception:
            continue

        file_name = image_mapping.get(image_id, f"unknown_{image_id}.png")
        cat_id = int(det.get("category_id", -1))
        cat_name = coco_category_name(cat_id)

        bbox = det.get("bbox", [])
        if not bbox or len(bbox) != 4:
            continue

        # 已是 [x1, y1, x2, y2]，无需转换
        x1, y1, x2, y2 = bbox
        if x2 <= x1 or y2 <= y1:
            continue  # 跳过异常框

        score = float(det.get("score", 0.0))
        area = abs((x2 - x1) * (y2 - y1))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": cat_id,
            "category_name": cat_name,
            "bbox": [x1, y1, x2, y2],
            "score": score,
            "area": area,
            "model": model_name,
            "version": VERSION
        })

    # ✅ 在保存前排序
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    # 输出保存
    output_path = os.path.join(os.path.dirname(INPUT_PATH), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    save_preview(model_name, output_path)
    print("=== 转换完成 ===")

if __name__ == "__main__":
    main()


=== Deformable-DETR → 标准化转换开始 ===


标准化中: 100%|██████████| 28800/28800 [00:00<00:00, 365800.05it/s]


✅ deformable-detr: 28800 条记录 → /opt/data/private/BlackBox/save/attack/detection/deformable-detr/ori/res-std.json
--------------------------------------------------
📄 deformable-detr - res-std.json 前 30 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      405.9403076171875,
      223.2398681640625,
      576.6092529296875,
      699.6097412109375
    ],
    "score": 0.8941296339035034,
    "area": 81301.54381155968,
    "model": "deformable-detr",
    "version": "ori"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      319.32122802734375,
      268.987060546875,
      445.9931945800781,
      683.2125244140625
    ],
    "score": 0.6872233152389526,
    "area": 52470.75410427526,
... (已省略更多行)
--------------------------------------------------
=== 转换完成 ===


# detr

In [25]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# ==================================================
# 配置
# ==================================================
VERSION = "ori"  # 原始检测版本
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco/annotations/instances_val2017.json"
INPUT_PATH = f"{BASE_DIR}/save/attack/detection/detr/ori/res.json"

# ==================================================
# 辅助函数
# ==================================================
def load_image_mapping(annotations_path):
    """加载 image_id → file_name 映射"""
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return {img["id"]: img["file_name"] for img in data["images"]}

def coco_category_name(cat_id: int) -> str:
    """COCO 类别名映射"""
    names = {
        1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane",
        6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light",
        11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench",
        16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow",
        22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
        27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase",
    }
    return names.get(cat_id, f"cls_{cat_id}")

def save_preview(model_name, output_path, max_lines=30):
    """打印前 max_lines 行 JSON 内容供人工验证"""
    print("-" * 50)
    print(f"📄 {model_name} - {os.path.basename(output_path)} 前 {max_lines} 行内容:")
    with open(output_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f.readlines()[:max_lines]):
            print(line.rstrip("\n"))
    print("-" * 50)

# ==================================================
# 主逻辑
# ==================================================
def main():
    print("=== DETR → 标准化转换开始 ===")

    if not os.path.exists(INPUT_PATH):
        print(f"⚠️ 未找到输入文件: {INPUT_PATH}")
        return

    image_mapping = load_image_mapping(ANNOT_PATH)
    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        detections = json.load(f)

    model_name = "detr"
    results = []

    for det in tqdm(detections, desc="标准化中"):
        image_id = int(det.get("image_id", -1))
        file_name = image_mapping.get(image_id, f"unknown_{image_id}.png")
        cat_id = int(det.get("category_id", -1))
        cat_name = coco_category_name(cat_id)

        bbox = det.get("bbox", [])
        if not bbox or len(bbox) != 4:
            continue

        x1, y1, x2, y2 = bbox
        # 修正异常框
        if x2 <= x1 or y2 <= y1:
            continue

        score = float(det.get("score", 0.0))
        area = abs((x2 - x1) * (y2 - y1))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": cat_id,
            "category_name": cat_name,
            "bbox": [x1, y1, x2, y2],
            "score": score,
            "area": area,
            "model": model_name,
            "version": VERSION
        })

    # ✅ 排序：image_id 升序，类别升序，score 降序
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    # ✅ 输出保存
    output_path = os.path.join(os.path.dirname(INPUT_PATH), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    save_preview(model_name, output_path)
    print("=== 转换完成 ===")

if __name__ == "__main__":
    main()


=== DETR → 标准化转换开始 ===


标准化中: 100%|██████████| 28800/28800 [00:00<00:00, 230336.74it/s]


✅ detr: 28800 条记录 → /opt/data/private/BlackBox/save/attack/detection/detr/ori/res-std.json
--------------------------------------------------
📄 detr - res-std.json 前 30 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      320.9579772949219,
      221.1162109375,
      572.7330322265625,
      705.0264892578125
    ],
    "score": 0.9996926784515381,
    "area": 121836.53690608218,
    "model": "detr",
    "version": "ori"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      201.8668670654297,
      464.63885498046875,
      241.6165008544922,
      505.5433044433594
    ],
    "score": 0.9438455700874329,
    "area": 1625.9368864931166,
--------------------------------------------------
=== 转换完成 ===
